# IMPORT

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

print(tf.__version__)

1.0.1


# PATH & PARAMETER

In [5]:
#KITTI DATASET 
TRAIN_DATA_PATH = '/data/training'
TEST_DATA_PATH =  '/data/testing'

IMAGE_WIDTH = 1248
IMAGE_HEIGHT = 384
IMAGE_CHANNEL = 3
BATCH_SIZE = 20
LR = 0.01
LR_DACAY_FACTOR = 0.1
LR_DECAY_STEPS = 10000
WEIGHT_DECAY = 0.0001
MAX_GRAD_NORM = 1.0

LOSS_COEF_CLS = 1.0
LOSS_COEF_BBOX = 5.0
LOSS_COEF_CONF_POS = 75.0
LOSS_COEF_CONF_NEG = 100.0

DEBUG_MODE = True
LOAD_PRETRAINED_MODEL = False
PRETRAINED_MODEL_PATH = ''
DATA_AUGMENTATION = True

EXCLUDE_HARD_EXAMPLES = False
NUM_THREAD = 8
QUEUE_CAPAITY = 100
IS_TRAINING = True

MOMENTUM = 0.9
EPSILON = 1e-16
BATCH_NORM_EPSILON = 1e-5

CLASS_NAMES = ('car', 'pedestrian', 'cyclist')
CLASS_NUM = len(CLASS_NAMES)


def set_anchors():
    H, W, B = 24, 78, 9
    anchor_shapes = np.reshape( #9개 앵커를 24 * 78개의 그리드에 
      [np.array(
          [[  36.,  37.], [ 366., 174.], [ 115.,  59.],
           [ 162.,  87.], [  38.,  90.], [ 258., 173.],
           [ 224., 108.], [  78., 170.], [  72.,  43.]])] * H * W,
      (H, W, B, 2)
    )
    center_x = np.reshape(
      np.transpose(
          np.reshape(
              np.array([np.arange(1, W+1)*float(IMAGE_WIDTH)/(W+1)]*H*B), 
              (B, H, W)
          ),
          (1, 2, 0)
      ),
      (H, W, B, 1)
    )
    center_y = np.reshape(
      np.transpose(
          np.reshape(
              np.array([np.arange(1, H+1)*float(IMAGE_HEIGHT)/(H+1)]*W*B),
              (B, W, H)
          ),
          (2, 1, 0)
      ),
      (H, W, B, 1)
    )
    anchors = np.reshape(
      np.concatenate((center_x, center_y, anchor_shapes), axis=3),
      (-1, 4)
    )

    return anchors

ANCHOR_BOX = set_anchors()
ANCHORS = len(ANCHOR_BOX)
ANCHOR_PER_GRID = 9

print('config done')


#그 외에 필요할만한 파라미터들은 이후에 만들면서 정의하도록 하자. 
NUM_TRAIN_DATA = 7481
NUM_EPOCH = 500
NUM_BATCHES_PER_EPOCH = NUM_TRAIN_DATA / NUM_EPOCH
NUM_TEST_DATA = 7518 # ls -al | wc -l로 셈. . .. total도 카운트되서 3개 뺌
NUM_TEST_PER_EPOCH = NUM_TEST_DATA / NUM_EPOCH

config done


# UTIL FUNCTION

In [6]:
#iou 및 corrdinates 변환 등등

def iou(box1, box2):
    lr = min(box1[0] + 0.5*box1[2], box2[0] + 0.5*box2[2]) - \
        max(box1[0] - 0.5*box1[2], box2[0] - 0.5*box2[2])
    
    if lr > 0: #겹치면
        tb = min(box1[1] + 0.5*box1[3], box2[1] + 0.5*box2[3]) - \
            max(box1[1] - 0.5*box1[3], box2[1] - 0.5*box2[3])
        if tb > 0: #상하도 겹치면
            intersection = tb * lr
            union = box1[2]*box1[3] + box2[2]*box2[3] - intersection
            
            return intersection / union
    return 0

def batch_iou(boxes, box): #박스 배치들과 하나의 박스 iou계산. 앵커 vs GT 
    lr = np.maximum(np.minimun(boxes[:,0] + 0.5*boxes[:,2], box[0] + 0.5*box[2]) - \
                   np.maximum(boxes[:,0] - 0.5*boxes[:,2], box[0] - 0.5*box[2]),
                   0)
    tb = np.maximum(np.minimun(boxes[:,1] + 0.5*boxes[:,3], box[1] + 0.5*box[3]) - \
                   np.maximum(boxes[:,1] - 0.5*boxes[:,3], box[1] - 0.5*box[3]),
                   0)
    intersection = lr * tb
    union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - intersection
    
    return intersection / union
    

def bbox_transform(bbox): # center -> minmax
    with tf.variable_scope('bbox_transform') as scope:
        cx, cy, w, h = bbox
        out_box = [[]]*4
        out_box[0] = cx-w/2
        out_box[1] = cy-h/2
        out_box[2] = cx+w/2
        out_box[3] = cy+h/2

# INPUT DATA

In [ ]:
# kitti dataset에 맞게 그냥 kitti로 해보자. 


# OPERATION

# TRAIN